### Import libraries

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image


### Read and Process text

In [2]:
# Read, then decode for py2 compat.
data ='short_jokes.txt'
text = open(data, 'rb').read().decode(encoding = "ISO-8859-1")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 859062 characters


In [3]:
# Take a look at the first 1000 characters in text
print(text[:1000])

I was eating at an Indian restaurant when a homeless man came in and asked the girl behind the counter if they had any food he could have. She told him there was Naan. [me narrating a documentary about narrators] "I can't hear what they're saying cuz I'm talking"
I was going to organize a space-themed birthday party for my son... ...but I couldn't planet. Telling my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vampires... men.
Squared My dick is X squared and I rooted your mom. Now she's my X I've been going through a really rough period at work this week It's my own fault for swapping my tampax for sand paper.
I call my dick fun Because it's what girls just want to have! If I could have dinner with anyone, dead or alive... ...I would choose alive. -B.J. Novak-
Be careful when you ROFL! I once heard a joke in a scissor factory... It left me in stitches. Two guys walk into a bar. The third guy ducks.
I'll bet Vampire Kiddies enjoy scabs as m

In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

94 unique characters


### Vectorize the text

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [6]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [7]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'I was eating ' ---- characters mapped to int ---- > [42  1 88 66 84  1 70 66 85 74 79 72  1]


### Create training examples and targets

In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

I
 
w
a
s


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'I was eating at an Indian restaurant when a homeless man came in and asked the girl behind the counte'
'r if they had any food he could have. She told him there was Naan. [me narrating a documentary about '
'narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\nI was going to organize a space-themed '
"birthday party for my son... ...but I couldn't planet. Telling my daughter garlic is good for you. Go"
'od immune system and keeps pests away.Ticks, mosquitos, vampires... men.\nSquared My dick is X squared'


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'I was eating at an Indian restaurant when a homeless man came in and asked the girl behind the count'
Target data: ' was eating at an Indian restaurant when a homeless man came in and asked the girl behind the counte'


In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 42 ('I')
  expected output: 1 (' ')
Step    1
  input: 1 (' ')
  expected output: 88 ('w')
Step    2
  input: 88 ('w')
  expected output: 66 ('a')
Step    3
  input: 66 ('a')
  expected output: 84 ('s')
Step    4
  input: 84 ('s')
  expected output: 1 (' ')


### Create training batches


In [13]:
# Setup batch size 
BATCH_SIZE = 64


steps_per_epoch = examples_per_epoch//BATCH_SIZE

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
#Use GPU
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  rnn2 = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),

    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [17]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

## Check the model

In [18]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 94) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           24064     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 94)            96350     
Total params: 4,058,718
Trainable params: 4,058,718
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([22, 59,  5, 45, 73, 49, 69, 72, 74, 37, 75, 74, 43, 87, 72, 79, 72,
       16, 40, 80,  1, 50,  8, 84, 57, 38, 77, 89, 42, 45, 69, 49, 36, 42,
       43, 43, 66,  7, 20, 33, 23, 90, 22, 83, 22, 18, 63, 26, 41, 59, 63,
       62, 22, 31, 91, 31, 62,  9, 63, 56, 88, 35, 86, 46, 30, 89, 90, 18,
       68, 25, 24, 67, 49, 81, 64, 27, 68, 49, 64, 37,  8, 22, 76, 35, 63,
       54, 67, 78, 75,  8,  8, 45, 89, 20, 72, 58, 86, 82, 23, 93])

## Train the model

In [22]:
#define a loss function
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 94)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.544327


In [23]:
#compile the model
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

### Configure checkpoints

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [25]:
EPOCHS=300

In [26]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/300
134/134 [==============================] - 8s 58ms/step - loss: 2.9542
Epoch 2/300
134/134 [==============================] - 6s 43ms/step - loss: 2.2269
Epoch 3/300
134/134 [==============================] - 6s 42ms/step - loss: 1.9823
Epoch 4/300
134/134 [==============================] - 6s 43ms/step - loss: 1.8146
Epoch 5/300
134/134 [==============================] - 6s 42ms/step - loss: 1.6927
Epoch 6/300
134/134 [==============================] - 6s 43ms/step - loss: 1.6021
Epoch 7/300
134/134 [==============================] - 6s 42ms/step - loss: 1.5301: 0s - loss
Epoch 8/300
134/134 [==============================] - 6s 44ms/step - loss: 1.4679
Epoch 9/300
134/134 [==============================] - 6s 43ms/step - loss: 1.4136
Epoch 10/300
134/134 [==============================] - 6s 44ms/step - loss: 1.3605
Epoch 11/300
134/134 [==============================] - 6s 43ms/step - loss: 1.3106
Epoch 12/300
134/134 [==============================] - 6s 43ms/step - los

## Generate text

### Restore the latest checkpoint

In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training/ckpt_300'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            24064     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 94)             96350     
Total params: 4,058,718
Trainable params: 4,058,718
Non-trainable params: 0
_________________________________________________________________


### Generate text

In [30]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 5000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
        
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"I"))


I jusud" - out the long everyo ficanys eat is ligh shople when I aver to fammate gerso when my searyon bering a need a just fucks but slien. . ..." Me: Son, my gette wead boul for any take tent int that re's greverd a quillol the corsid do you. dit get tunar!.
TILF into say "Lewriggs Become chone is 60 I tellings that don't cat. Theey ononshiy hear read a comperied it twe lo that tak say to eationshis to preett? A daysmistingering into a backentig, bett us use pleed alyally Ifprenito and wealratess fat' What daing Rambuatbody's take had got thers reame iibem to tock? So I wate ao wen othit of goeras a blon..... a hemphe. Irully is a cop hands and water call dat ? Like manttier the doninger. Why do weam in COVRECLERRRYOURY
Mas mon, sex.
I'm theers King! Clus caund wtill light in Kittay.... Opurried.
Whate a nost parkl makey dous the nose just and Wead. "whath ask bigfer houge. I wantees dond.
What's is can'ta the are at the ould was us-use cuss. I sand alwa. Stank the say. . why Nond Ra